In [1]:
# Загружаем необходимые библиотеки

import datetime                  # Пакет для работы с временными форматами
import pickle                    # Пакет для подгрузки данных специфического для питона формата
import requests                  # Пакет для скачки данных из этих ваших интернетов
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import time

from tqdm import tqdm_notebook

In [151]:
# мой номер странички
my_user_id = 389809125
# версия используемого API
version = '5.103'
# подгружаем токен
with open('token.txt') as f:
    token = f.read()

In [3]:
url = "https://api.vk.com/method/users.get?user_ids={user_id}&access_token={token}&scope=offline&v={version}"
url = url.format(user_id=my_user_id, token=token, version=version)

In [4]:
response = requests.get(url)
response

<Response [200]>

In [5]:
name = response.json()
name

{'response': [{'id': 389809125,
   'first_name': 'Дарья',
   'last_name': 'Астахова',
   'can_access_closed': True,
   'is_closed': False}]}

In [6]:
def vkDownload(method, parameters, token=token, version=version):
    """
        Возвращает результат запроса по методу

        method: string
            метод из документации, который хотим использовать

        parameters: string
            параметры используемого метода

        token: string
            токен Oauth доступа

        version: string
            версия API
    """

    # составляем ссылку
    url = 'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
    url = url.format(method=method, parameters=parameters, token=token, version=version)
    # запрашиваем ссылку и переводим в json (словарь)
    response = requests.get(url).json()

    return response

# Парсим группы

In [7]:
# Основные параметры, которые мы хотим получить
fields = 'screen_name, city, home_town, sex, education, bdate'

Делаем словарь из списка самых популярных групп на религиозную тему

In [153]:
relig_groups = {
     'Храм-на-Крови' : 'xram_na_krovi',
     'Смоленский храм и Часовня Ксении Блаженной' : 'smolhramspb',
     '12 Храмов. Cовpеменный взгляд на Пpавославие' : '12hramov'
    }

In [157]:
def getGroupMembers(group_id):
    """
        Возвращает список всех пользователей данной группы
        Итерации идут батчами по 1000 пользователей

        group_id: string
            идентификатор группы (ссылка)
    """

    # Узнаём число запросов, которое надо сделать
    count = vkDownload('groups.getMembers','group_id=' + group_id)['response']['count']

    # выясняем, сколько запросов нам понадобиться
    n = int(np.ceil(count/1000))

    # вектор, где мы будем хранить id пользователей
    members = []

    for i in tqdm_notebook(range(n)):
        # при помощи метода groups.getMembers получаем пользователей группы
        current_members = vkDownload('groups.getMembers','group_id='+group_id+'&offset='+str(1000*i))
        members.extend(current_members['response']['items'])
        # перед следующим запросом немножко подождем
        time.sleep(0.4)

    return members

In [41]:
# Цикл для выгрузки всех участников групп
# Выкачиваем членов каждой группы

relig_people = {}
for relig, group_id in relig_groups.items():
    relig_people[relig] = getGroupMembers(group_id)

# Сохраняем словарик с id пользователей
with open('relig_people', 'wb') as f:
    pickle.dump(relig_people, f)

<ipython-input-40-665e950aa616>:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n)):


  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/52 [00:00<?, ?it/s]

In [42]:
for relig, people in relig_people.items():
    print("Группа храма: {}, число подписчиков: {}".format(relig, len(people)))

Группа храма: Храм-на-Крови, число подписчиков: 7078
Группа храма: Смоленский храм и Часовня Ксении Блаженной, число подписчиков: 32630
Группа храма: 12 Храмов. Cовpеменный взгляд на Пpавославие, число подписчиков: 51692


Теперь мы знаем количество подписчиков в каждой группе

In [43]:
count = sum(relig_people.values(), [])
count

[59997,
 130858,
 154018,
 293055,
 362872,
 373797,
 378717,
 403337,
 416505,
 417933,
 433027,
 475328,
 575200,
 581840,
 622093,
 627413,
 654023,
 666568,
 671829,
 701921,
 702696,
 719329,
 724827,
 735855,
 739887,
 848261,
 976180,
 989295,
 1009305,
 1017765,
 1035411,
 1049726,
 1154012,
 1178856,
 1204102,
 1226574,
 1243376,
 1273553,
 1291611,
 1309678,
 1323035,
 1363605,
 1377328,
 1403532,
 1432596,
 1442406,
 1463235,
 1507987,
 1604795,
 1606044,
 1607329,
 1631466,
 1635979,
 1690997,
 1743772,
 1756160,
 1763461,
 1784544,
 1796060,
 1867176,
 1895411,
 1908996,
 1990992,
 2002468,
 2013556,
 2089308,
 2110761,
 2168176,
 2183907,
 2188093,
 2295657,
 2297776,
 2374451,
 2383217,
 2458071,
 2479676,
 2491701,
 2499391,
 2504974,
 2505253,
 2507291,
 2513905,
 2518916,
 2542420,
 2598885,
 2612060,
 2706101,
 2706688,
 2727310,
 2754528,
 2765588,
 2796235,
 2826035,
 2833694,
 2854271,
 2867542,
 2897452,
 2901440,
 2907888,
 2925483,
 2928147,
 2931344,
 2955788,

Составим список из уникальных пользователей

In [45]:
# объединяем всех подписчиков в один лист
unique_people = sum(relig_people.values(), [])
# берем только уникальных людей
unique_people = list(set(unique_people))

len(unique_people)

88585

88585 уникальных пользователей, подписанных на группы храмов. Теперь проведем статистичесский анализ

Для выкачки соц-дема будем использовать метод users.get. За раз можно скачать информацию о 100 пользователях

In [161]:
def getUserInformation():
    """
        Возвращает словарь с выгруженными социально-демографическими данными
        по всем пользователям, подписанным на группы
    """

    # инициализируем пустой словарь, где будут хранитсья пользователи с их соцдемом
    relig_people_information = {}
    # проходимся по всем кандидатам
    for relig, people in relig_people.items():
        # текущий список пользователей с соц-демом
        current_info = []
        #посчитаем число пользователей для группы
        count = len(people)
        # Выяснили число запросов
        n = int(np.ceil(count/500))

        for i in tqdm_notebook(range(n)):
            # берем текущий срез подписчиков
            ids = people[i*500:(i+1)*500]
            ids = ','.join([str(user) for user in ids])
            # выгружаем их и берем данные
            info = vkDownload('users.get','user_ids={}&fields={}'.format(ids, fields))
            info = info['response']
            # записываем в текущий лист
            current_info.extend(info)
            # ждем перед следующим запросом
            time.sleep(0.4)

        # записываем в финальный словарь
        relig_people_information[relig] = current_info

    return relig_people_information

In [119]:
relig_people_information = getUserInformation()

<ipython-input-118-855186f09891>:18: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(n)):


  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/104 [00:00<?, ?it/s]

In [162]:
#спрячем данные в дамп pickle
with open('relig_people_information', 'wb') as f:
     pickle.dump(relig_people_information, f)


Преобразуем полученный словарь в удобный датафрейм для последующей работы

In [163]:
import pandas as pd
import numpy as np

In [164]:
# Инициализация пустого датафрейма
relig_information_df = pd.DataFrame()

# Обход всех кандидатов и их соц-дем данных
for relig, people in relig_people_information.items():
    # Преобразование списка людей в датафрейм
    df = pd.DataFrame(people)

    # Добавляем столбцы 'city' и 'relig'
    df['city'] = df['city'].apply(lambda x: x['title'] if x is not np.nan else np.nan)
    df['church'] = relig

    # Объединяем датафреймы с помощью concat
    relig_information_df = pd.concat([relig_information_df, df], ignore_index=True)

In [165]:
relig_information_df

,id,bdate,city,sex,screen_name,first_name,last_name,can_access_closed,is_closed,university,university_name,faculty,faculty_name,graduation,education_form,education_status,home_town,deactivated,church
0,59997,2.1.1988,Москва,1,yugeniyavasilieva88,Евгения,Васильева,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови
1,130858,NaN,Москва,1,d.syuzeva,Дарья,Сюзёва,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови
2,154018,26.10.1986,Москва,2,id154018,Андрей,Афанасьев,True,False,238.0,МИРЭА (до 2015),948.0,Институт информационных технологий,2009.0,Очное отделение,Выпускник (специалист),Белокаменная,NaN,Храм-на-Крови
3,293055,30.11.1987,Санкт-Петербург,2,alex_ivshukov,Александр,Ившуков,True,False,1.0,СПбГУ,2129210.0,Филологический,2010.0,Очное отделение,Студент (бакалавр),"Если Петербург - культурная столица России, то...",NaN,Храм-на-Крови
4,362872,NaN,Санкт-Петербург,1,id362872,Лариса,Тоноян,True,False,1.0,СПбГУ,16.0,Институт философии (бывш. Философский факультет),1980.0,Очное отделение,Кандидат наук,"Ереван, Санкт-Петербург",NaN,Храм-на-Крови
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91395,1030751389,2.6.1981,NaN,2,id1030751389,Павел,Карасев,True,False,0.0,,0.0,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие
91396,1031173365,5.2.1998,Мытищи,2,id1031173365,Артём,Неверов,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие
91397,1032174923,1.3.1980,NaN,1,id1032174923,Надя,Иванова,True,False,0.0,,0.0,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие
91398,1035367021,7.1.1977,NaN,1,id1035367021,Таня,Павлова,True,False,0.0,,0.0,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие


In [166]:
relig_information_df.shape

(91400, 19)

91400 всего подписчиков в совокупности у трех групп, при этом мы знаем, что 88585 уникальных. Делаем вывод, что в основном подписчики уникальны и не повторяются, однако 2815 пользователей подписаны на несколько пабликов одновременно.

# Преобразование данных

In [167]:
relig_information_df.to_csv('relig-nonofficial.csv', sep=',')

In [168]:
relig_information_df['sex'] = np.where(relig_information_df['sex']==1, 'Female', 'Male')
relig_information_df['sex']

,sex
0,Female
1,Female
2,Male
3,Male
4,Female
...,...
91395,Male
91396,Male
91397,Female
91398,Female


In [169]:
relig_information_df

,id,bdate,city,sex,screen_name,first_name,last_name,can_access_closed,is_closed,university,university_name,faculty,faculty_name,graduation,education_form,education_status,home_town,deactivated,church
0,59997,2.1.1988,Москва,Female,yugeniyavasilieva88,Евгения,Васильева,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови
1,130858,NaN,Москва,Female,d.syuzeva,Дарья,Сюзёва,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови
2,154018,26.10.1986,Москва,Male,id154018,Андрей,Афанасьев,True,False,238.0,МИРЭА (до 2015),948.0,Институт информационных технологий,2009.0,Очное отделение,Выпускник (специалист),Белокаменная,NaN,Храм-на-Крови
3,293055,30.11.1987,Санкт-Петербург,Male,alex_ivshukov,Александр,Ившуков,True,False,1.0,СПбГУ,2129210.0,Филологический,2010.0,Очное отделение,Студент (бакалавр),"Если Петербург - культурная столица России, то...",NaN,Храм-на-Крови
4,362872,NaN,Санкт-Петербург,Female,id362872,Лариса,Тоноян,True,False,1.0,СПбГУ,16.0,Институт философии (бывш. Философский факультет),1980.0,Очное отделение,Кандидат наук,"Ереван, Санкт-Петербург",NaN,Храм-на-Крови
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91395,1030751389,2.6.1981,NaN,Male,id1030751389,Павел,Карасев,True,False,0.0,,0.0,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие
91396,1031173365,5.2.1998,Мытищи,Male,id1031173365,Артём,Неверов,True,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие
91397,1032174923,1.3.1980,NaN,Female,id1032174923,Надя,Иванова,True,False,0.0,,0.0,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие
91398,1035367021,7.1.1977,NaN,Female,id1035367021,Таня,Павлова,True,False,0.0,,0.0,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие


## Определение возвраста пользователя по дате рождения

In [170]:
relig_information_df['bdate'].str.split('.', expand=True)
#агрумент expand=True значит создание новых столбцов из разденного

,0,1,2
0,2,1,1988
1,NaN,NaN,NaN
2,26,10,1986
3,30,11,1987
4,NaN,NaN,NaN
...,...,...,...
91395,2,6,1981
91396,5,2,1998
91397,1,3,1980
91398,7,1,1977


In [171]:
df_bdate = relig_information_df
#датафрейм df_ нужен для временного хранения данных о дате рождения в разных столбцах
df_ = df_bdate['bdate'].str.split('.', expand=True)
#объединяем два датарфрейиа по index
df_bdate = pd.concat([df_bdate, df_], axis=1)
df_bdate

,id,bdate,city,sex,screen_name,first_name,last_name,can_access_closed,is_closed,university,...,faculty_name,graduation,education_form,education_status,home_town,deactivated,church,0,1,2
0,59997,2.1.1988,Москва,Female,yugeniyavasilieva88,Евгения,Васильева,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови,2,1,1988
1,130858,NaN,Москва,Female,d.syuzeva,Дарья,Сюзёва,False,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови,NaN,NaN,NaN
2,154018,26.10.1986,Москва,Male,id154018,Андрей,Афанасьев,True,False,238.0,...,Институт информационных технологий,2009.0,Очное отделение,Выпускник (специалист),Белокаменная,NaN,Храм-на-Крови,26,10,1986
3,293055,30.11.1987,Санкт-Петербург,Male,alex_ivshukov,Александр,Ившуков,True,False,1.0,...,Филологический,2010.0,Очное отделение,Студент (бакалавр),"Если Петербург - культурная столица России, то...",NaN,Храм-на-Крови,30,11,1987
4,362872,NaN,Санкт-Петербург,Female,id362872,Лариса,Тоноян,True,False,1.0,...,Институт философии (бывш. Философский факультет),1980.0,Очное отделение,Кандидат наук,"Ереван, Санкт-Петербург",NaN,Храм-на-Крови,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91395,1030751389,2.6.1981,NaN,Male,id1030751389,Павел,Карасев,True,False,0.0,...,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,2,6,1981
91396,1031173365,5.2.1998,Мытищи,Male,id1031173365,Артём,Неверов,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,5,2,1998
91397,1032174923,1.3.1980,NaN,Female,id1032174923,Надя,Иванова,True,False,0.0,...,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,1,3,1980
91398,1035367021,7.1.1977,NaN,Female,id1035367021,Таня,Павлова,True,False,0.0,...,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,7,1,1977


In [172]:
df_bdate = df_bdate.rename(columns={0: 'day_bdate', 1: 'month_bdate', 2: 'year_bdate'})

In [173]:
df_bdate

,id,bdate,city,sex,screen_name,first_name,last_name,can_access_closed,is_closed,university,...,faculty_name,graduation,education_form,education_status,home_town,deactivated,church,day_bdate,month_bdate,year_bdate
0,59997,2.1.1988,Москва,Female,yugeniyavasilieva88,Евгения,Васильева,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови,2,1,1988
1,130858,NaN,Москва,Female,d.syuzeva,Дарья,Сюзёва,False,True,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови,NaN,NaN,NaN
2,154018,26.10.1986,Москва,Male,id154018,Андрей,Афанасьев,True,False,238.0,...,Институт информационных технологий,2009.0,Очное отделение,Выпускник (специалист),Белокаменная,NaN,Храм-на-Крови,26,10,1986
3,293055,30.11.1987,Санкт-Петербург,Male,alex_ivshukov,Александр,Ившуков,True,False,1.0,...,Филологический,2010.0,Очное отделение,Студент (бакалавр),"Если Петербург - культурная столица России, то...",NaN,Храм-на-Крови,30,11,1987
4,362872,NaN,Санкт-Петербург,Female,id362872,Лариса,Тоноян,True,False,1.0,...,Институт философии (бывш. Философский факультет),1980.0,Очное отделение,Кандидат наук,"Ереван, Санкт-Петербург",NaN,Храм-на-Крови,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91395,1030751389,2.6.1981,NaN,Male,id1030751389,Павел,Карасев,True,False,0.0,...,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,2,6,1981
91396,1031173365,5.2.1998,Мытищи,Male,id1031173365,Артём,Неверов,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,5,2,1998
91397,1032174923,1.3.1980,NaN,Female,id1032174923,Надя,Иванова,True,False,0.0,...,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,1,3,1980
91398,1035367021,7.1.1977,NaN,Female,id1035367021,Таня,Павлова,True,False,0.0,...,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,7,1,1977


In [174]:

#удаляем столбцы с днем и месяцем рождения, нам нужен только год
df_bdate = df_bdate.drop(['day_bdate', 'month_bdate'], axis=1)


In [175]:
df_bdate.isnull().sum()

,0
id,0
bdate,24037
city,27934
sex,0
screen_name,0
first_name,0
last_name,0
can_access_closed,0
is_closed,0
university,59829


Удаляем все строки, если в столбце year_bdate NaN

In [176]:
df_bdate = df_bdate.dropna(subset=['year_bdate'])

In [177]:
df_bdate['year_bdate'] = pd.to_numeric(df_bdate['year_bdate'], errors= 'coerce')

In [178]:
df_bdate['age'] = 2024 - df_bdate['year_bdate']

Теперь мы получили возвраст и дату рождения в формате "года"

In [179]:
df_bdate

,id,bdate,city,sex,screen_name,first_name,last_name,can_access_closed,is_closed,university,...,faculty,faculty_name,graduation,education_form,education_status,home_town,deactivated,church,year_bdate,age
0,59997,2.1.1988,Москва,Female,yugeniyavasilieva88,Евгения,Васильева,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови,1988,36
2,154018,26.10.1986,Москва,Male,id154018,Андрей,Афанасьев,True,False,238.0,...,948.0,Институт информационных технологий,2009.0,Очное отделение,Выпускник (специалист),Белокаменная,NaN,Храм-на-Крови,1986,38
3,293055,30.11.1987,Санкт-Петербург,Male,alex_ivshukov,Александр,Ившуков,True,False,1.0,...,2129210.0,Филологический,2010.0,Очное отделение,Студент (бакалавр),"Если Петербург - культурная столица России, то...",NaN,Храм-на-Крови,1987,37
7,403337,10.11.1987,Екатеринбург,Male,sharafulinden,Denis,Shevchenko,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови,1987,37
10,433027,13.6.1984,Санкт-Петербург,Female,sablinaiv,Ирина,Поликарпова,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Храм-на-Крови,1984,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91394,1028557845,17.1.1991,NaN,Male,id1028557845,Роман,Каплунов,True,False,0.0,...,0.0,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,1991,33
91395,1030751389,2.6.1981,NaN,Male,id1030751389,Павел,Карасев,True,False,0.0,...,0.0,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,1981,43
91396,1031173365,5.2.1998,Мытищи,Male,id1031173365,Артём,Неверов,True,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,1998,26
91397,1032174923,1.3.1980,NaN,Female,id1032174923,Надя,Иванова,True,False,0.0,...,0.0,,0.0,NaN,NaN,,NaN,12 Храмов. Cовpеменный взгляд на Пpавославие,1980,44


# Анализ результатов

In [139]:
df_bdate.groupby(
    by=['church'])[['age']].mean().sort_values(by='age', ascending=False)

,age
church,
Смоленский храм и Часовня Ксении Блаженной,44.808030
12 Храмов. Cовpеменный взгляд на Пpавославие,43.451593
Храм-на-Крови,42.410636


Средний возвраст во всех сообществах примерно одинаковый и состовляет в среднем 42, 43 и 44 года. Разница чемжу минимальным и макимальным значением: 2 года.

In [183]:
df_bdate['sex'].value_counts()

,count
sex,
Female,27058
Male,12418


In [186]:
(27058 - 12418)


14640

Мы можем заметить сильный перекос в сторону женской аудитории. Разница составляет 14640 человек

In [188]:
( 14640/27058)*100

54.10599453026831

На 54% женщин больше, чем мужчин

In [140]:
df_bdate.groupby(
    by=['sex'])[['age']].mean().sort_values(by='age', ascending=False)

,age
sex,
Female,44.967256
Male,41.244001


Средний возвраст у женщин интересующихся сообществами на религиозную тематику выше примерно на 4 года, чем у мужчин (44.9 и 41.2 соответвенно)

**Место жительства**

In [148]:
relig_information_df['city'].unique()

array(['Москва', 'Санкт-Петербург', 'Североморск', ...,
       'Петрово (Угранское с/п)', 'Makao', 'п/л Зеленый Гай'],
      dtype=object)

In [149]:
df['city'].nunique()

2493

In [189]:
2493-1125

1368

Количество городов очень велико. Их количество означает, что подписчиками являются не только граждане России (так как в России 1125 годов). Более того, большая часть наодится не России.

In [150]:
df_bdate.groupby(
    by=['city'])[['id']].count().sort_values(by='id', ascending=False).head(100)

,id
city,
Санкт-Петербург,5030
Москва,2885
Екатеринбург,1167
Нижний Новгород,403
Воронеж,393
...,...
Сызрань,30
Киев,29
Новороссийск,29


Самое большое количество подписчиков проживает в городах милионников, котрые входят в топ-4 самых густонаселенных городов: Санкт-Петербург, Москва, Екатеринбург.